Equipe: Leila K.Silva, Jane S.Deutsch, Jaqueline Damacena Duarte, Marli Aparecida Silva, Michael Abraao Soares Miranda

### Conceitos de uma Convolutional Neural Network
Uma rede convolucional - CNN ou ConvNet, é uma classe de rede neural artificial do tipo feed-forward.  
A arquitetura de uma ConvNet faz uma analogia ao padrão de conectividade de Neurônios no Cérebro Humano e foi inspirada na organização do Visual Cortex. Os neurônios individuais respondem a estímulos apenas em uma região restrita do campo visual conhecida como Campo Receptivo. Uma coleção desses campos são sobreposto para cobrir toda a área visual. Assim, a rede convolucional calcula um novo valor para um pixel da imagem com base nos pixels da vizinhança. 

Redes neurais convolucionais são muito adequadas em detectar padrões em imagens pois elas conseguem aprender invariância do ponto de vista. Ou seja, podem detectar os padrões independentemente da posição deles na imagem, uma vez que o filtro é aplicado em toda a imagem.

Arquitetura : MobileNetV2: A arquitetura do MobileNetV2 é baseada em uma estrutura residual invertida onde a entrada e saída do bloco residual são camadas finas de gargalo opostas aos modelos residuais tradicionais que usam representações expandidas na entrada do MobileNetV2 usa convoluções leves para filtrar recursos na camada de expansão intermediária
Utilizamos um pré-treino para transferencia de aprendizado a partir de pesos conhecidos da base Imagenet.

Depois, executamos uma rede neural convolucional do tipo Stack of Layers ( Keras.Sequential) com 1 camada densa com a função de não linearidade sigmoid e utilizando maxpooling para nossa base.


Camadas de pooling = Camadas de agrupamento de pixels, ou pooling,  reduzem as dimensões dos dados, combinando as saídas de clusters de neurônios em uma camada em um único neurônio na próxima camada. O pooling pode ser local, ou global, máximo ou médio. Foi escolhido o Global Maxpooling 2D do Keras, que usa o valor máximo de cada cluster de neurônios na camada anterior e atua em todos os neuronios da camada convolucional.

Funções de Ativação/não linearidade = escolhemos a função sigmoid ou logística.

Filtros - Um filtro pode ser visto como uma abstração de um foco quadriculado que desliza por toda a imagem produzindo uma saída para a próxima camada. A saida de cada foco por onde o filtro é aplicado forma um pixel do mapa representativo da imagem, que é então passado à próxima camada. Essa operação de deslizar uma grade de parâmetros pela imagem é chamada de convolução. 

Camada Densa - são camadas totalmente conectadas (Fully Connected- FC), 
Transfer Learning : O aprendizado por transferência acelera o treinamento, pois permite reutilizar os modelos de classificação de imagens que já foram previamente treinados, apenas treinando novamente a camada superior da rede que determina as classes às quais uma imagem pode pertencer.

Data Augmentation: é uma técnica para criar novos dados de treinamento a partir de dados de treinamento existentes a partir da criação de versões de imagens do dataset de treinamento que pertencem à mesma classe da imagem original. No trabalho utilizamos apenas alguns procedimentos básicos, no caso de ajuste de brilho e normalização da imagem e flip, mas observamos que não favoreceu o modelo.

Próximos passos, pretendemos trabalhar hyperparametros, tuning e buscar outros testes com outras augmentations para ver se favorece o modelo.

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.display import Image, display
from sklearn.model_selection import GridSearchCV


In [ ]:
tf.version

In [ ]:
train_csv = pd.read_csv('../input/train.csv')
print(train_csv.describe())
print(train_csv.head())

In [ ]:
# Has Cactus
display(Image('../input/train/train/0004be2cfeaba1c0361d39e2b000257b.jpg'))
display(Image('../input/train/train/000c8a36845c0208e833c79c1bffedd1.jpg'))

# No cactus
display(Image('../input/train/train/ffede47a74e47a5930f81c0b6896479e.jpg'))
display(Image('../input/train/train/fff43acb3b7a23edcc4ae937be2b7522.jpg'))

## Preparação da base de treino e de validação 

#### Base de treino corresponde a  90% dos arquivos e de teste10%

In [ ]:
filenames = ['../input/train/train/' + fname for fname in train_csv['id'].tolist()]
labels = train_csv['has_cactus'].tolist()


train_filenames, test_filenames, train_labels, test_labels = train_test_split(filenames,
                                                                            labels,
                                                                            train_size=0.9,
                                                                            random_state=420)

size_train = len(train_filenames)
size_test = len(test_filenames)

### Fazendo leitura/carregamento das imagens e redefinindo o tamanho para trabalhar
#### Normaliza e padroniza o tamanho da imagem

In [ ]:
IMAGE_SIZE = 96 
BATCH_SIZE = 32

def _parse_fn(filename, label):
  image_decoded = tf.image.decode_jpeg(tf.io.read_file(filename))
  image_normalized = (tf.cast(image_decoded, tf.float32)/127.5) - 1
  image_resized = tf.image.resize(image_normalized, (IMAGE_SIZE, IMAGE_SIZE))
  #image_resized = _augment_(image_resized) 
  return image_resized, label

In [ ]:
def _augment_(image):
       image = tf.image.random_brightness(image,255.0, 1)
       image = tf.clip_by_value(image, 0.0, 255.0)
       image = tf.image.rot90(x, tf.random_uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
       return image
    

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((tf.constant(train_filenames),tf.constant(train_labels))).map(_parse_fn).shuffle(buffer_size=10000).batch(BATCH_SIZE)
test_data = tf.data.Dataset.from_tensor_slices((tf.constant(test_filenames), tf.constant(test_labels))).map(_parse_fn).batch(BATCH_SIZE)


### Modelo (pré-)treinado com pesos conhecidos ( Pesos do imagenet)

In [ ]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

modelMNV2 = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights='imagenet')
modelMNV2.trainable = False


In [ ]:
pool_layer = tf.keras.layers.GlobalMaxPooling2D()
dense_layer = tf.keras.layers.Dense(1, activation='relu')

In [ ]:
model = tf.keras.Sequential([modelMNV2,pool_layer,dense_layer])
model.compile(optimizer= 'Adam',loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

### Treinando o modelo com na base aerial cactus

In [ ]:
num_epochs = 30
steps_per_epoch = int(size_train)
val_steps = 10


In [ ]:
history = model.fit(train_data.repeat(),
                    epochs=num_epochs,
                    steps_per_epoch = steps_per_epoch,
                    validation_data=test_data.repeat(), 
                    validation_steps=val_steps)

In [ ]:
model.save_weights('weights_epoch_30.h5')


In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
test_folder = "../input/test/"
test_datagen = ImageDataGenerator(
    rescale=1. / 127.5)

test_generator = test_datagen.flow_from_directory(
    directory=test_folder,
    target_size=(96,96),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

In [ ]:
pred=model.predict_generator(test_generator,verbose=1)
pred_binary = [0 if value<0.50 else 1 for value in pred]  

In [ ]:
# aplicando a base de teste
csv_file = open("submission.csv","w")
csv_file.write("id,has_cactus\n")
for filename, prediction in zip(test_generator.filenames,pred_binary):
    name = filename.split("/")[1]
    csv_file.write(str(name)+","+str(prediction)+"\n")
csv_file.close()

In [ ]:
name

In [ ]:
# base de teste do treino
#pred = model.predict(test_data, batch_size=32, verbose=1, steps=55)
#pred_binary =np.ravel([0 if value<0.50 else 1 for value in pred])  
#submission_df = pd.DataFrame({'id':test_filenames,'has_cactus':pred_binary})
#submission_df.to_csv('testedotreino.csv', index=False)
#pred_binary

# Referências
#### Easy Image Classification with TensorFlow 2.0: TOWARDS DATA SCIENCE . Disponível em: https://towardsdatascience.com/easy-image-classification-with-tensorflow-2-0-f734fee52d13

#### A comprehensive guide to convolutional neural networkds. TOWARDS DATA SCIENCE. Disponível em : https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53

#### WIKIPEDIA. Convolutional Neural Network https://en.wikipedia.org/wiki/Convolutional_neural_network#Max_pooling_shape

#### GITHUB/ Matheus Facure: Resolvendo CAPTCHAs com Redes Neurais Convolucionais. https://matheusfacure.github.io/2017/03/12/cnn-captcha/

#### Frlemarchand: Simple-cnn-using-keras. (Kernel participante da competição). https://www.kaggle.com/frlemarchand/simple-cnn-using-keras